In [1]:
!pip install tensorflow

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.


In [2]:
!pip install git+https://github.com/tflearn/tflearn.git 

  Cloning https://github.com/tflearn/tflearn.git to c:\users\grant\appdata\local\temp\pip-req-build-4v9xr29b
  Running setup.py bdist_wheel for tflearn: started
  Running setup.py bdist_wheel for tflearn: finished with status 'done'
  Stored in directory: C:\Users\grant\AppData\Local\Temp\pip-ephem-wheel-cache-x0gxk87i\wheels\5a\18\2a\c62b9937b37223da935fb6b2965f02fbc45691e460a08b91b4
Successfully built tflearn


notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.


In [3]:
# Do this in a separate python interpreter session, since you only have to do it once
import nltk
nltk.download('punkt')

# Do this in your ipython notebook or analysis script
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\grant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tensorflow as tf
import tflearn
import random

C:\Users\grant\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [5]:
# import our chat-bot intents file
import json

f = open('c:/Users/grant/desktop/AiLawMiniCourseHW/book.txt', 'r')
s = f.read()

intents = json.loads(s)
print(intents)

#with open('c:/Users/grant/desktop/AiLawMiniCourseHW/book.txt', 'r') as book_data:
#    json_data = json.load(book_data)
#    intents = json.load(json_data)

{'intents': [{'tag': 'greating', 'patterns': ['Hello', 'How are you doing?', 'Good Morning', 'Good Afternoon'], 'responses': ['Hello!', 'Good to see you!'], 'context_set': ''}, {'tag': 'types', 'patterns': ['What types of damages are available for this case?', 'What types of damages are available to the plaintiff?', 'What types of damages are available in this case?'], 'responses': ['Typically damages are calculate in general and special in California', 'California provides special and general damages to plaintiffs']}, {'tag': 'general', 'patterns': ['Can you explain general damages?', 'Explain general damages to the jury?', 'What is the meaning of general damages?'], 'responses': ['General damages are non-economic damages suffered by the plaintiff', 'General damages are defined as non-economic damages']}, {'tag': 'special', 'patterns': ['Can you explain special damages?', 'Explain special damages to the jury?', 'What is the meaning of special damages?'], 'responses': ['Special damages

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

36 documents
14 classes ['calculated', 'construction', 'double', 'economic', 'general', 'greating', 'medical', 'multiplier', 'noneconomic', 'pain', 'single', 'special', 'total', 'types']
59 unique stemmed words ['a', 'afternoon', 'and', 'ar', 'avail', 'aw', 'award', 'be', 'believ', 'calc', 'can', 'cas', 'construct', 'cost', 'dam', 'do', 'doing', 'doubl', 'econom', 'explain', 'for', 'gen', 'good', 'hello', 'how', 'in', 'includ', 'is', 'jury', 'mean', 'med', 'morn', 'much', 'multiply', 'noneconom', 'of', 'pain', 'plaintiff', 'pric', 'rebuild', 'requir', 'should', 'singl', 'spec', 'suff', 'that', 'the', 'thi', 'think', 'to', 'tot', 'trail', 'typ', 'wer', 'what', 'wid', 'would', 'yo', 'you']


In [7]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [8]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.07850 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.07850 - acc: 0.9997 -- iter: 32/36
Training Step: 5000  | total loss: 0.07657 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.07657 - acc: 0.9997 -- iter: 36/36
--
INFO:tensorflow:C:\Users\grant\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
['calculated', 'construction', 'double', 'economic', 'general', 'medical', 'multiplier', 'noneconomic', 'pain', 'single', 'special', 'total', 'types']


In [11]:
print(model.predict([p]))

[[0.01620386 0.14024475 0.18542093 0.02958297 0.01148112 0.04139716
  0.19776902 0.03262744 0.05329175 0.14057268 0.05986594 0.07605327
  0.0154891 ]]


In [12]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )